#### Find the average sales price of each zone in the dataset.

In [1]:
# Grab the data from the FINAL DATA DIR
import os
import sys
# Get the current working directory
current_directory = os.getcwd()
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
os.chdir(parent_directory)

from src.paths import *
import pandas as pd
import numpy as np

# Set PD display options to show all columns and column width
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

#heavy_manufacturing = pd.read_csv(LAND_USE_DIR / 'Complete_Heavy_Manufacturing_Codes.csv', index_col = [0])
# get recent transactions
SL = pd.read_csv(FRONTEND_DIR / 'sales_list_frontend.csv', index_col = [0])
LR = pd.read_csv(FRONTEND_DIR / 'local_roll_frontend.csv', index_col = [0])
DS = pd.read_csv(FRONTEND_DIR / 'secured_basic_frontend.csv', index_col = [0])
LU = pd.read_csv(FRONTEND_DIR / 'land_use_frontend.csv', index_col = [0])
HZ = pd.read_csv(FRONTEND_DIR / 'hazards_frontend.csv', index_col = [0])

C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_40032\2307088051.py:23: DtypeWarning: Columns (145) have mixed types. Specify dtype option on import or set low_memory=False.
  DS = pd.read_csv(FRONTEND_DIR / 'secured_basic_frontend.csv', index_col = [0])


In [3]:
def find_mixed_data_types_and_nulls(df):
    mixed_types_info = {}
    null_values_info = {}

    for column in df.columns:
        # Get unique data types in the column
        unique_types = df[column].map(type).unique()
        
        # Check for mixed data types
        if len(unique_types) > 1:
            type_details = {}
            for t in unique_types:
                example = df[column][df[column].apply(lambda x: isinstance(x, t))].iloc[0]
                type_details[str(t)] = example
            mixed_types_info[column] = type_details
        
        # Check for null values
        null_count = df[column].isnull().sum()
        if null_count > 0:
            null_values_info[column] = null_count
    
    results = {}
    
    if mixed_types_info:
        results['Mixed Data Types'] = mixed_types_info
    
    if null_values_info:
        results['Null Values'] = null_values_info
    
    if not results:
        return "The DataFrame has no columns with mixed types or null values."
    
    return results

def validate_ain_column(df, column_name='assessor_identification_number'):
    # Check for negative numbers
    if (df[column_name] < 0).any():
        raise ValueError(f"The column '{column_name}' contains negative numbers.")
    
    # Check for 10 digits length
    if not df[column_name].astype(str).str.len().eq(10).all():
        raise ValueError(f"The column '{column_name}' does not have all entries with 10 digits.")
    
    # Check for duplicates
    if df[column_name].duplicated().any():
        raise ValueError(f"The column '{column_name}' contains duplicate values.")
    
    # Return "The DataFrame does not contain any issues with the AIN column."
    return "The DataFrame does not contain any issues with the AIN column."

In [4]:
find_mixed_data_types_and_nulls(DS)

{'Mixed Data Types': {'trust_names': {"<class 'str'>": 'PATEL TRUST',
   "<class 'float'>": nan},
  'impairment_description': {"<class 'float'>": nan,
   "<class 'str'>": 'Land Physical Impaired'}},
 'Null Values': {'trust_names': 1648861, 'impairment_description': 2421397}}

In [5]:
# create a temporary dataframe with the address and ain 

,mapbook_number,page_number,parcel_number,assessor_identification_number,taxrate_area,agency_number,land_roll_year,land_current_value,imp_current_roll_year,imp_current_value,situs_house_no,street_name,unit,city,zip,first_owner_name,tax_stat_key,year_sold_to_state,zoning_code,use_code,partial_interest,doc_reason_code,exemption_type,pp_key,pp_value,pp_exemption_val,fixture_val,fixture_exemption_val,num_howmowner_exemption,homeowner_exemption_val,real_estate_exemption_val,last_sale_verif_key,last_sale_amount,sale_two_verif_key,sale_two_amount,sale_three_verif_key,sale_three_amount,building_subpart_1,building_design_1,building_quality_1,building_year_built_1,building_units_1,building_bedrooms_1,building_baths_1,building_square_feet_1,building_subpart_2,building_design_2,building_quality_2,building_year_built_2,building_units_2,building_bedrooms_2,building_baths_2,building_square_feet_2,building_subpart_3,building_design_3,building_quality_3,building_year_built_3,building_units_3,building_bedrooms_3,building_baths_3,building_square_feet_3,building_subpart_4,building_design_4,building_quality_4,building_year_built_4,building_units_4,building_bedrooms_4,building_baths_4,building_square_feet_4,building_subpart_5,building_design_5,building_quality_5building_year_built_5,building_units_5,building_bedrooms_5,building_baths_5,building_square_feet_5,bd5_square_feet,legal_first_line,legal_second_line,legal_third_line,legal_fourth_line,legal_fifth_line,legal_last_line,land_base_year,imp_base_year,land_base_val,imp_base_val,cluster_location,cluster_type,cluster_appraisal_unit,land_reason_key,impairment_key,ddt_amount,bd1_year_change,building_unit_cost_1,building_rcn_main_1,building_year_change_2,building_unit_cost_2,building_rcn_main_2,building_year_change_3,building_unit_cost_3,building_rcn_main_3,building_year_change_4,building_unit_cost_4,building_rcn_main_4,building_year_change_5,building_unit_cost_5,building_rcn_main_5,landlord_reappraisal_year,landlord_units,first_transfree_name,first_transfree_overflow,second_transfree_name,document_key,document_number,county_name,state_name,trust_names,trust_indicator,care_of,recording_date_year,recording_date_month,recording_date_day,last_sale_date_year,last_sale_date_month,last_sale_date_day,sale_two_date_year,sale_two_date_month,sale_two_date_day,sale_three_date_year,sale_three_date_month,sale_three_date_day,average_unit_rent,average_single_family_sale,average_multi_family_sale,average_commercial_sale,average_industrial_sale,average_condominium_sale,residential_zone_price,commercial_zone_price,industrial_zone_price,mixed_use_zone_price,market_comparable_city,market_comparable,impairment_description,tax_status
0,2004,1,3,2004001003,16,0,2024,711689,2024,301176,8321,FAUST AVE,Unknown,LOS ANGELES,91304,PATEL HANISH B AND ALPA P,0,0,LARE9,0101,0,U,10,0,0,0,0,0,0,0,0,1,9,0,0,0,0,101,131,0,1973,1,4,3,2090,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,TRACT NO 25040 LOT 99,Unknown,Unknown,Unknown,Unknown,Unknown,2006,2006,600254,252282,2,1,21,0,0,9,1978,2379,49720,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,Unknown,0,1,798640,Los Angeles,California,PATEL TRUST,1,0,2009,5,29,2005,9,6,2003,8,26,2000,1,11,2250,900000,1300000,2000000,3000000,500000,800000,1000000,1200000,900000,LOS ANGELES,Above Average for Single Family Homes,NaN,Taxes paid not delinquent
1,2004,1,4,2004001004,16,0,2024,370538,2024,255879,8313,FAUST AVE,Unknown,LOS ANGELES,91304,ALI SYED SHAH AND DILRUBA S,0,0,LARE9,0101,0,U,10,0,0,0,0,0,0,0,0,1,9,0,0,0,0,101,131,0,1973,1,5,3,2479,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,TRACT NO 25040 LOT 100,Unknown,Unknown,Unknown,Unknown,Unknown,2010,2010,292800,202200,2,1,21,0,0,9,1978,2346,58160,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,Unknown,0,1,1775715,Los Angeles,California,ALI FAMILY TRUST,1,0,2021,12,1,2009,10,19,2006,1,5,1993,3,16,2250,900000,1300000,2000000,3000000,500000,800000,1000000,1200000,900000,LOS ANGELES,Above Average for Single Family Homes,NaN,Taxes paid n

In [31]:
# Save the data to the frontend directory
DS.to_csv(FRONTEND_DIR / 'secured_basic_final.csv')
LR.to_csv(FRONTEND_DIR / 'local_roll_final.csv')
HZ.to_csv(FRONTEND_DIR / 'hazards_final.csv')
LU.to_csv(FRONTEND_DIR / 'land_use_final.csv')
SL.to_csv(FRONTEND_DIR / 'sales_list_final.csv')